# Root Cause Analysis - PyRCA

## Import all the necessary libraries

In [53]:
import networkx as nx
import numpy as np
import pyrca

from pyrca.analyzers.ht import HT, HTConfig
from pyrca.analyzers.epsilon_diagnosis import EpsilonDiagnosis, EpsilonDiagnosisConfig
from pyrca.analyzers.bayesian import BayesianNetwork, BayesianNetworkConfig
from pyrca.analyzers.random_walk import RandomWalk, RandomWalkConfig
from pyrca.analyzers.rcd import RCD, RCDConfig

import matplotlib.pyplot as plt
import pandas as pd
import random
import os

# Some functions and libraries throw warnings
from sklearn.exceptions import ConvergenceWarning

import warnings
warnings.filterwarnings('ignore', category=RuntimeWarning)
warnings.filterwarnings('ignore', category=ConvergenceWarning)
warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=FutureWarning)


## Custom Functions

### Data and Directory Functions

In [54]:
def draw_save(G_graph,pos,node_colors,file_name:str, inter_type:str):
    test = 'test' # no drawings
    plt.figure(figsize=(12, 10))
    plt.xlim((-12,14))
    plt.ylim((-12,8))
    plt.title(f'Causal Graph: {inter_type}', fontsize=12)
    nx.draw(G_graph, pos,with_labels=True,node_size=2000, node_color=[node_colors[node] for node in G_graph.nodes()], font_size=6, arrowsize=8,width=0.5)
    # plt.savefig(file_name)
    # nx.write_gml(G_graph, f'{file_name[:-4]}.gml')

def get_file_name(var:str):
    return var.rsplit('-', 1)[0]

def create_data_from_list(files):
    data_df = pd.DataFrame()
    for file in files:
        to_join_df = file
        data_df = pd.concat([data_df,to_join_df.loc[to_join_df.index]],ignore_index=True)
    return data_df

def create_train_data(files,startrow):
    data_df = pd.DataFrame()
    for file_key, file_path in files.items():
        to_join_df =  pd.read_csv(file_path, skiprows=range(1, startrow))
        data_df = pd.concat([data_df,to_join_df.loc[to_join_df.index]],ignore_index=True)
    return data_df

def get_from_dir(directory_path):
    # Get a list of all items (files and directories) in the specified path
    all_items = os.listdir(directory_path)
    # Iterate over each item and check if it's a directory
    for item in all_items:
        folder_path = os.path.join(directory_path, item)
        if os.path.isdir(folder_path):
            print(f'\nFolder: {item}')
        

def get_from_folder(directory_path,folder):
    files = {}
    folder_path = os.path.join(directory_path, folder)
    if os.path.isdir(folder_path):
        print(f'Selected folder: {folder}')
        folder_contents = os.listdir(folder_path)
        for file in folder_contents:
            file_path = os.path.join(folder_path, file)
            print(f'-> {file}')
            files[f'{file}'] = file_path
    return files, folder_path

def results_top_3(results,abnormal_set):
    result_columns = results.columns.to_list()
    result_index = results.index.to_list()

    result_data_3top = pd.DataFrame(columns=result_index, index=result_columns)

    for col in result_columns:
        for ind in result_index:
            for elem in results[col][ind]:
                if elem in abnormal_set[col.rsplit('-', 1)[0]].to_list():
                    result_data_3top[ind][col] = 1

    result_data_3top = result_data_3top.fillna(0)

    total_hit = (result_data_3top.sum('index')/len(result_columns)).to_dict()
    result_data_3top = pd.concat([result_data_3top,pd.DataFrame(total_hit,index=['Total'])])

    return result_data_3top

def results_top_1(results,abnormal_set):
    result_columns = results.columns.to_list()
    result_index = results.index.to_list()
    result_data_1top = pd.DataFrame(columns=result_index, index=result_columns)
    for col in result_columns:
        for ind in result_index:
            try: elem = results[col][ind][0] # In case where RCD did not provide any root causes
            except: continue
            if elem in abnormal_set[col.rsplit('-', 1)[0]].to_list():
                result_data_1top[ind][col] = 1

    result_data_1top = result_data_1top.fillna(0)

    total_hit = (result_data_1top.sum('index')/len(result_columns)).to_dict()
    result_data_1top = pd.concat([result_data_1top,pd.DataFrame(total_hit,index=['Total'])])

    return result_data_1top


### Algorithms

#### HT - Algorithms

In [55]:
# Function run hypothesis testing algorithm
def run_HT(folder_path: str,
           files: dict,
           startrow: int,
           normal_data: pd.DataFrame,
           nodes: list,
           edges_list:list,
           key_nodes: list,
           colors: dict,
           pos: dict):
    
    G_graph = nx.DiGraph()
    G_graph.add_nodes_from(nodes)
    G_graph.add_edges_from(edges_list)  # Make sure `edges_list` is defined somewhere

    adj_matrix_extended_pd = nx.to_pandas_adjacency(G_graph, nodes)
    
    interventions = {'gripper_1':'interGripper1',
                    'gripper_2':'interGripper2',
                    'max_Vel_1':'interVeloRob1',
                    'max_Vel_2':'interVeloRob2',
                    'camera_1':'interCamera1',
                    'camera_2':'interCamera2',
                    'camera_3':'interCamera3',
                    'conveyor_1':'interConveyor1',
                    'conveyor_2':'interConveyor2',
                    'conveyor_3':'interConveyor3',
                    'feeder_1':'interFeeder1',
                    'feeder_2':'interFeeder2',
                    'feeder_3':'interFeeder3',
                    'size_1':'interSize1',
                    'size_2':'interSize2',
                    'size_3':'interSize3'}
    
    normal_data_df = normal_data
    abnormal_files = {}
    file_names = []
    counter = 0
    
    for file_key, file_path in files.items():       
        temp_file_key = file_key.rsplit('-', 1)[0]
        if temp_file_key in interventions:
            counter += 1
            abnormal_files[file_key] = file_path
            file_names.append(temp_file_key+'-'+str(counter))
        
    abnormal_paths_df = pd.DataFrame.from_dict(abnormal_files, orient='index', columns=['file_path'])
    results_out = pd.DataFrame(index = ['HT'])

    for file_counter, abnormal_file_path in enumerate(abnormal_paths_df.values.flatten()):
        abnormal_data_df = pd.read_csv(abnormal_file_path, skiprows=range(1, startrow))
        abnormal_data_df = abnormal_data_df[nodes]

        model = HT(config=HTConfig(adj_matrix_extended_pd))
        model.train(normal_data_df)

        abnormal_nodes = []
        new_colors = colors.copy()

        root_cause_results = []

        results = pd.DataFrame()
        for node in key_nodes:
            if (abnormal_data_df[node] <100).any(): # Score instead of EoL
                abnormal_nodes.append(node)
                new_colors[node] = 'yellow'
                results[node] = model.find_root_causes(abnormal_data_df, node, True).to_list()
        
        rank1_root_cause = []
        rank2_root_cause = []
        rank3_root_cause = []

        for node in abnormal_nodes:
            rank1_root_cause.append(results[node][0]['root_cause'])
            rank2_root_cause.append(results[node][1]['root_cause'])
            rank3_root_cause.append(results[node][2]['root_cause'])

            root_cause_results.append(results[node][0]['root_cause'])
            root_cause_results.append(results[node][1]['root_cause'])
            root_cause_results.append(results[node][2]['root_cause'])

        results_out[file_names[file_counter]] = [root_cause_results]

    return results_out

#### ED - Algorithms

In [56]:
# Function run epsilon diagnosis algorithm
def run_ED(folder_path: str,
           files: dict,
           startrow: int,
           normal_data: pd.DataFrame,
           nodes: list,
           edges_list:list,
           key_nodes: list,
           colors: dict,
           pos: dict):
    
    G_graph = nx.DiGraph()
    G_graph.add_nodes_from(nodes)
    G_graph.add_edges_from(edges_list)  # Make sure `edges_list` is defined somewhere

    adj_matrix_extended_pd = nx.to_pandas_adjacency(G_graph, nodes)
    
    interventions = {'gripper_1':'interGripper1',
                    'gripper_2':'interGripper2',
                    'max_Vel_1':'interVeloRob1',
                    'max_Vel_2':'interVeloRob2',
                    'camera_1':'interCamera1',
                    'camera_2':'interCamera2',
                    'camera_3':'interCamera3',
                    'conveyor_1':'interConveyor1',
                    'conveyor_2':'interConveyor2',
                    'conveyor_3':'interConveyor3',
                    'feeder_1':'interFeeder1',
                    'feeder_2':'interFeeder2',
                    'feeder_3':'interFeeder3',
                    'size_1':'interSize1',
                    'size_2':'interSize2',
                    'size_3':'interSize3'}
    
    normal_data_df = normal_data
    abnormal_files = {}
    file_names = []
    counter = 0

    for file_key, file_path in files.items():       
        temp_file_key = file_key.rsplit('-', 1)[0]
        if temp_file_key in interventions:
            counter += 1
            abnormal_files[file_key] = file_path
            file_names.append(temp_file_key+'-'+str(counter))
    
    abnormal_paths_df = pd.DataFrame.from_dict(abnormal_files, orient='index', columns=['file_path'])

    results_out = pd.DataFrame(index = ['ED'])

    for file_counter, abnormal_file_path in enumerate(abnormal_paths_df.values.flatten()):
        abnormal_data_df = pd.read_csv(abnormal_file_path, skiprows=range(1, startrow))
        abnormal_data_df = abnormal_data_df[nodes]

        model = EpsilonDiagnosis(EpsilonDiagnosisConfig(alpha=0.05,root_cause_top_k=3,bootstrap_time=200))
        abnormal_nodes = []
        new_colors = colors.copy()
        root_cause_results = []

        model.train(normal_data_df)

        for node in key_nodes:
            # Basic Anomaly Detection
            if (abnormal_data_df[node] <100).any():
                abnormal_nodes.append(node)
                new_colors[node] = 'yellow'

        results = model.find_root_causes(abnormal_data_df).to_list()
        
        rank1_root_cause = []
        rank2_root_cause = []
        rank3_root_cause = []

        for i in range(0,int(np.trunc(len(results)/3))):
            rank1_root_cause.append(results[i]['root_cause'])
            rank2_root_cause.append(results[i+1]['root_cause'])
            rank3_root_cause.append(results[i+2]['root_cause'])

            root_cause_results.append(results[0]['root_cause'])
            root_cause_results.append(results[1]['root_cause'])
            root_cause_results.append(results[2]['root_cause'])

        results_out[file_names[file_counter]] = [root_cause_results]


    return results_out


#### RW - Algorithms

In [57]:
# Function run random walk algorithm
def run_RW(folder_path: str,
           files: dict,
           startrow: int,
           normal_data: pd.DataFrame,
           nodes: list,
           edges_list:list,
           key_nodes: list,
           colors: dict,
           pos: dict):
    
    G_graph = nx.DiGraph()
    G_graph.add_nodes_from(nodes)
    G_graph.add_edges_from(edges_list)  # Make sure `edges_list` is defined somewhere

    adj_matrix_extended_pd = nx.to_pandas_adjacency(G_graph, nodes)
    adj_matrix_extended = nx.adjacency_matrix(G_graph,nodes).todense()
    adjacency_df = pd.DataFrame(adj_matrix_extended, index=G_graph.nodes(), columns=G_graph.nodes())
    
    interventions = {'gripper_1':'interGripper1',
         'gripper_2':'interGripper2',
         'max_Vel_1':'interVeloRob1',
         'max_Vel_2':'interVeloRob2',
         'camera_1':'interCamera1',
         'camera_2':'interCamera2',
         'camera_3':'interCamera3',
         'conveyor_1':'interConveyor1',
         'conveyor_2':'interConveyor2',
         'conveyor_3':'interConveyor3',
         'feeder_1':'interFeeder1',
         'feeder_2':'interFeeder2',
         'feeder_3':'interFeeder3',
         'size_1':'interSize1',
         'size_2':'interSize2',
         'size_3':'interSize3'}
    
    normal_data_df = normal_data
    abnormal_files = {}
    file_names = []
    counter = 0

    for file_key, file_path in files.items():
        temp_file_key = file_key.rsplit('-', 1)[0]
        if temp_file_key in interventions:
            counter += 1
            abnormal_files[file_key] = file_path
            file_names.append(temp_file_key+'-'+str(counter))
    
    abnormal_paths_df = pd.DataFrame.from_dict(abnormal_files, orient='index', columns=['file_path'])

    results_out = pd.DataFrame(index = ['RW'])

    for file_counter, abnormal_file_path in enumerate(abnormal_paths_df.values.flatten()):
        abnormal_data_df = pd.read_csv(abnormal_file_path, skiprows=range(1, startrow))
        abnormal_data_df = abnormal_data_df[nodes]

        model = RandomWalk(RandomWalkConfig(graph=adjacency_df,root_cause_top_k=3))
        abnormal_nodes = []
        new_colors = colors.copy()
        root_cause_results = []

        for node in key_nodes:
            # Basic Anomaly Detection
            if (abnormal_data_df[node] <100).any(): # from EoL to score
                abnormal_nodes.append(node)
                new_colors[node] = 'yellow'

        results = model.find_root_causes(abnormal_nodes,create_data_from_list([abnormal_data_df,normal_data_df])).to_list()

        rank1_root_cause = []
        rank2_root_cause = []
        rank3_root_cause = []
        

        for i in range(0,int(np.trunc(len(results)/3))):
            rank1_root_cause.append(results[i]['root_cause'])
            rank2_root_cause.append(results[i+1]['root_cause'])
            rank3_root_cause.append(results[i+2]['root_cause'])

            root_cause_results.append(results[0]['root_cause'])
            root_cause_results.append(results[1]['root_cause'])
            root_cause_results.append(results[2]['root_cause'])

        results_out[file_names[file_counter]] = [root_cause_results]

    return results_out


#### RCD - Algorithms

In [58]:
# Function run Root cause discovery algorithm
def run_RCD(folder_path: str,
           files: dict,
           startrow: int,
           normal_data: pd.DataFrame,
           nodes: list,
           edges_list:list,
           key_nodes: list,
           colors: dict,
           pos: dict):
    
    G_graph = nx.DiGraph()
    G_graph.add_nodes_from(nodes)
    G_graph.add_edges_from(edges_list)  # Make sure `edges_list` is defined somewhere

    adj_matrix_extended_pd = nx.to_pandas_adjacency(G_graph, nodes)
    adj_matrix_extended = nx.adjacency_matrix(G_graph,nodes).todense()
    adjacency_df = pd.DataFrame(adj_matrix_extended, index=G_graph.nodes(), columns=G_graph.nodes())
    
    interventions = {'gripper_1':'interGripper1',
         'gripper_2':'interGripper2',
         'max_Vel_1':'interVeloRob1',
         'max_Vel_2':'interVeloRob2',
         'camera_1':'interCamera1',
         'camera_2':'interCamera2',
         'camera_3':'interCamera3',
         'conveyor_1':'interConveyor1',
         'conveyor_2':'interConveyor2',
         'conveyor_3':'interConveyor3',
         'feeder_1':'interFeeder1',
         'feeder_2':'interFeeder2',
         'feeder_3':'interFeeder3',
         'size_1':'interSize1',
         'size_2':'interSize2',
         'size_3':'interSize3'}

    normal_data_df = normal_data
    abnormal_files = {}
    file_names = []
    counter = 0

    for file_key, file_path in files.items():
        temp_file_key = file_key.rsplit('-', 1)[0]        
        if temp_file_key in interventions:
            counter += 1
            abnormal_files[file_key] = file_path
            file_names.append(temp_file_key+'-'+str(counter))
            
    abnormal_paths_df = pd.DataFrame.from_dict(abnormal_files, orient='index', columns=['file_path']) 
    results_out = pd.DataFrame(index = ['RCD'])
    
    for file_counter, abnormal_file_path in enumerate(abnormal_paths_df.values.flatten()):
        abnormal_data_df = pd.read_csv(abnormal_file_path, skiprows=range(1, startrow))
        abnormal_data_df = abnormal_data_df[nodes]

        abnormal_nodes = []
        new_colors = colors.copy()
        root_cause_results = []

        results = pd.DataFrame()
        for node in key_nodes:
            model = RCD(RCDConfig(bins=5,f_node=node,localized=True,k=3)) #alpha_step=0.05,start_alpha=0.001,alpha_limit=0.5
            if (abnormal_data_df[node] < 100).any():
                error = True
                while error:
                    abnormal_nodes.append(node)
                    new_colors[node] = 'yellow'
                    try:
                        results[node] = model.find_root_causes(normal_data_df,abnormal_data_df).to_list()                        
                        error = False
                    except:
                        continue

            rank1_root_cause = []
            rank2_root_cause = []
            rank3_root_cause = []

            if len(results[node]) == 3:
                rank1_root_cause.append(results[node][0]['root_cause'])
                rank2_root_cause.append(results[node][1]['root_cause'])
                rank3_root_cause.append(results[node][2]['root_cause'])

                root_cause_results.append(results[node][0]['root_cause'])
                root_cause_results.append(results[node][1]['root_cause'])
                root_cause_results.append(results[node][2]['root_cause'])
            elif len(results[node]) == 2:
                rank1_root_cause.append(results[node][0]['root_cause'])
                rank2_root_cause.append(results[node][1]['root_cause'])

                root_cause_results.append(results[node][0]['root_cause'])
                root_cause_results.append(results[node][1]['root_cause'])

            elif len(results[node]) == 1:
                rank1_root_cause.append(results[node][0]['root_cause'])
                root_cause_results.append(results[node][0]['root_cause'])

        results_out[file_names[file_counter]] = [root_cause_results]

    return results_out


## Create the Causal Graph (non-lagged)

### Graph Parameters

In [59]:
nodes = [
    'cam_1_X', 'cam_2_X', 'cam_3_X',
    'cam_1_Y', 'cam_2_Y', 'cam_3_Y',
    'EoL_1_X', 'EoL_2_X', 'EoL_3_X', 'EoL_4_X', 'EoL_5_X', 'EoL_6_X',
    'EoL_1_Y', 'EoL_2_Y', 'EoL_3_Y', 'EoL_4_Y', 'EoL_5_Y', 'EoL_6_Y',
    'rob_1_1', 'rob_1_2', 'rob_1_3', 'rob_1_4', 'rob_1_maxVel',
    'rob_2_1', 'rob_2_2', 'rob_2_3', 'rob_2_4', 'rob_2_maxVel',
    'rob_1_supply', 'rob_2_supply',
    'rob_1_vacuum', 'rob_2_vacuum',
    'con_1','con_2','con_3',
    'score'
]


edges = [
    ('cam_1_X', 'rob_2_1'), ('cam_1_Y', 'rob_2_1'),
    ('cam_1_X', 'rob_2_2'), ('cam_1_Y', 'rob_2_2'),
    ('cam_1_X', 'rob_2_3'), ('cam_1_Y', 'rob_2_3'),
    ('cam_1_X', 'rob_2_4'), ('cam_1_Y', 'rob_2_4'),
    
    ('cam_2_X', 'rob_1_1'), ('cam_2_Y', 'rob_1_1'),
    ('cam_2_X', 'rob_1_2'), ('cam_2_Y', 'rob_1_2'),
    ('cam_2_X', 'rob_1_3'), ('cam_2_Y', 'rob_1_3'),
    ('cam_2_X', 'rob_1_4'), ('cam_2_Y', 'rob_1_4'),
    
    ('cam_3_X', 'rob_1_1'), ('cam_3_Y', 'rob_1_1'),
    ('cam_3_X', 'rob_1_2'), ('cam_3_Y', 'rob_1_2'),
    ('cam_3_X', 'rob_1_3'), ('cam_3_Y', 'rob_1_3'),
    ('cam_3_X', 'rob_1_4'), ('cam_3_Y', 'rob_1_4'),
    
    ('rob_1_maxVel', 'rob_1_1'), ('rob_1_maxVel', 'rob_1_2'),
    ('rob_1_maxVel', 'rob_1_3'), ('rob_1_maxVel', 'rob_1_4'),
    
    ('rob_2_maxVel', 'rob_2_1'), ('rob_2_maxVel', 'rob_2_2'),
    ('rob_2_maxVel', 'rob_2_3'), ('rob_2_maxVel', 'rob_2_4'),
    
    ('con_2', 'rob_1_1'), ('con_2', 'rob_1_2'), ('con_2', 'rob_1_3'), ('con_2', 'rob_1_4'),
    ('con_3', 'rob_1_1'), ('con_3', 'rob_1_2'), ('con_3', 'rob_1_3'), ('con_3', 'rob_1_4'),

    ('con_2', 'rob_2_1'), ('con_2', 'rob_2_2'), ('con_2', 'rob_2_3'), ('con_2', 'rob_2_4'),
    ('con_1', 'rob_2_1'), ('con_1', 'rob_2_2'), ('con_1', 'rob_2_3'), ('con_1', 'rob_2_4'),

    ('con_2', 'EoL_1_X'), ('con_2', 'EoL_1_Y'),
    
    ('rob_1_1', 'rob_2_1'), ('rob_1_1', 'rob_2_2'), ('rob_1_1', 'rob_2_3'), ('rob_1_1', 'rob_2_4'),
    ('rob_1_2', 'rob_2_1'), ('rob_1_2', 'rob_2_2'), ('rob_1_2', 'rob_2_3'), ('rob_1_2', 'rob_2_4'),
    ('rob_1_3', 'rob_2_1'), ('rob_1_3', 'rob_2_2'), ('rob_1_3', 'rob_2_3'), ('rob_1_3', 'rob_2_4'),
    ('rob_1_4', 'rob_2_1'), ('rob_1_4', 'rob_2_2'), ('rob_1_4', 'rob_2_3'), ('rob_1_4', 'rob_2_4'),

    ('rob_1_supply', 'rob_1_vacuum'), 
    ('rob_2_supply', 'rob_2_vacuum'),

    
    ('rob_1_vacuum', 'rob_2_1'), ('rob_1_vacuum', 'rob_2_2'),
    ('rob_1_vacuum', 'rob_2_3'), ('rob_1_vacuum', 'rob_2_4'),

    ('rob_1_1', 'EoL_2_X'), ('rob_1_2', 'EoL_2_X'),
    ('rob_1_3', 'EoL_2_X'), ('rob_1_4', 'EoL_2_X'),
    ('rob_1_1', 'EoL_2_Y'), ('rob_1_2', 'EoL_2_Y'),
    ('rob_1_3', 'EoL_2_Y'), ('rob_1_4', 'EoL_2_Y'),
    
    ('rob_2_1', 'EoL_3_X'), ('rob_2_2', 'EoL_3_X'),
    ('rob_2_3', 'EoL_3_X'), ('rob_2_4', 'EoL_3_X'),
    ('rob_2_1', 'EoL_3_Y'), ('rob_2_2', 'EoL_3_Y'),
    ('rob_2_3', 'EoL_3_Y'), ('rob_2_4', 'EoL_3_Y'),
    
    ('rob_2_1', 'EoL_4_X'), ('rob_2_2', 'EoL_4_X'),
    ('rob_2_3', 'EoL_4_X'), ('rob_2_4', 'EoL_4_X'),
    ('rob_2_1', 'EoL_4_Y'), ('rob_2_2', 'EoL_4_Y'),
    ('rob_2_3', 'EoL_4_Y'), ('rob_2_4', 'EoL_4_Y'),
    
    ('rob_2_1', 'EoL_5_X'), ('rob_2_2', 'EoL_5_X'),
    ('rob_2_3', 'EoL_5_X'), ('rob_2_4', 'EoL_5_X'),
    ('rob_2_1', 'EoL_5_Y'), ('rob_2_2', 'EoL_5_Y'),
    ('rob_2_3', 'EoL_5_Y'), ('rob_2_4', 'EoL_5_Y'),

    ('rob_2_1', 'EoL_6_X'), ('rob_2_2', 'EoL_6_X'),
    ('rob_2_3', 'EoL_6_X'), ('rob_2_4', 'EoL_6_X'),
    ('rob_2_1', 'EoL_6_Y'), ('rob_2_2', 'EoL_6_Y'),
    ('rob_2_3', 'EoL_6_Y'), ('rob_2_4', 'EoL_6_Y'),

    ('rob_1_vacuum', 'EoL_2_X'), ('rob_1_vacuum', 'EoL_2_Y'),
    
    ('rob_2_vacuum', 'EoL_3_X'), ('rob_2_vacuum', 'EoL_3_Y'),
    ('rob_2_vacuum', 'EoL_4_X'), ('rob_2_vacuum', 'EoL_4_Y'),
    ('rob_2_vacuum', 'EoL_5_X'), ('rob_2_vacuum', 'EoL_5_Y'),
    ('rob_2_vacuum', 'EoL_6_X'), ('rob_2_vacuum', 'EoL_6_Y'),

    ('EoL_1_X','score'), ('EoL_2_X','score'), ('EoL_3_X','score'), ('EoL_4_X','score'), ('EoL_5_X','score'), ('EoL_6_X','score'),
    ('EoL_1_Y','score'), ('EoL_2_Y','score'), ('EoL_3_Y','score'), ('EoL_4_Y','score'), ('EoL_5_Y','score'), ('EoL_6_Y','score')
]

print(len(edges))

check_nodes = ['score']


134


### Additional Parameters

In [60]:
pos = {
    'cam_1_X':(8,4), 'cam_2_X':(-9,6), 'cam_3_X':(-5,6),
    'cam_1_Y':(8,2), 'cam_2_Y':(-7,6), 'cam_3_Y':(-3,6),
    'EoL_1_X':(10,-8), 'EoL_2_X':(-10,-8), 'EoL_3_X':(-6,-8), 'EoL_4_X':(-2,-8), 'EoL_5_X':(2,-8), 'EoL_6_X':(6,-8),
    'EoL_1_Y':(12,-8), 'EoL_2_Y':(-8,-8), 'EoL_3_Y':(-4,-8), 'EoL_4_Y':(0,-8), 'EoL_5_Y':(4,-8), 'EoL_6_Y':(8,-8),
    'score':(0,-10),
    'rob_2_1':(-6,-4), 'rob_2_2':(-4,-4), 'rob_2_3':(-2,-4), 'rob_2_4':(-0,-4), 'rob_2_maxVel':(2,-4),
    'rob_1_1':(-9,1), 'rob_1_2':(-7,1), 'rob_1_3':(-5,1), 'rob_1_4':(-3,1), 'rob_1_maxVel':(-1,1),
    'rob_1_vacuum':(2,1), 'rob_2_vacuum':(5,-4),'rob_1_supply':(5,1), 'rob_2_supply':(8,-4),
    'con_1':(8,-1),'con_2':(11,6),'con_3':(3,6)
}

colors = {
    'cam_1_X':'skyblue', 'cam_2_X':'skyblue', 'cam_3_X':'skyblue',
    'cam_1_Y':'skyblue', 'cam_2_Y':'skyblue', 'cam_3_Y':'skyblue',
    'EoL_1_X':'lightgreen', 'EoL_2_X':'lightgreen', 'EoL_3_X':'lightgreen', 'EoL_4_X':'lightgreen', 'EoL_5_X':'lightgreen', 'EoL_6_X':'lightgreen',
    'EoL_1_Y':'lightgreen', 'EoL_2_Y':'lightgreen', 'EoL_3_Y':'lightgreen', 'EoL_4_Y':'lightgreen', 'EoL_5_Y':'lightgreen', 'EoL_6_Y':'lightgreen',
    'score':'lightsalmon',
    'rob_1_1':'tan', 'rob_1_2':'tan', 'rob_1_3':'tan', 'rob_1_4':'tan', 'rob_1_maxVel':'tan',
    'rob_2_1':'tan', 'rob_2_2':'tan', 'rob_2_3':'tan', 'rob_2_4':'tan', 'rob_2_maxVel':'tan',
    'rob_1_vacuum':'tan', 'rob_2_vacuum':'tan','rob_1_supply':'tan', 'rob_2_supply':'tan',
    'con_1':'lightgrey','con_2':'lightgrey','con_3':'lightgrey'
}

## Show datasets in folders

In [61]:
# Specify the directory path
directory_path = 'G:\\My Drive\\Master Thesis\\Simulation\\Dataset_single'
get_from_dir(directory_path)


Folder: Dataset_normal

Folder: Dataset_size_1

Folder: Dataset_feeder_3

Folder: Dataset_gripper_1

Folder: Dataset_max_Vel_2


In [62]:
folder_input = input('Please select dataset folder.')
files,folder_path = get_from_folder(directory_path,folder_input)

Selected folder: Dataset_max_Vel_2
-> max_Vel_2-1.csv
-> max_Vel_2-2.csv
-> max_Vel_2-3.csv
-> max_Vel_2-4.csv
-> max_Vel_2-5.csv
-> max_Vel_2-6.csv
-> max_Vel_2-7.csv
-> max_Vel_2-8.csv
-> max_Vel_2-9.csv
-> max_Vel_2-10.csv
-> max_Vel_2-11.csv
-> max_Vel_2-12.csv
-> max_Vel_2-13.csv
-> max_Vel_2-14.csv
-> max_Vel_2-15.csv
-> max_Vel_2-16.csv


In [63]:
files_normal,folder_path_normal = get_from_folder(directory_path,'Dataset_normal')
for file_key, file_path in files_normal.items():
    temp_file_key = file_key.rsplit('-', 1)[0]
    if temp_file_key == 'normal':
        normal_data_df = pd.read_csv(file_path, skiprows=range(1, 839))
        normal_data_df = normal_data_df[nodes]

Selected folder: Dataset_normal
-> normal-1.csv
-> normal-2.csv
-> normal-3.csv
-> normal-4.csv
-> normal-5.csv
-> normal-6.csv
-> normal-7.csv
-> normal-8.csv


## Run RCA trough folder

A warm-up phase is considered. Each product takes roughly 26.30 seconds from the assembly entry point to the assembly end of line. There is some distance from the drop point of the items to the start and some interventions need time to become stable, thus we need to skip in total roughly 839 rows. Each 8 seconds a new product exits the assembly line.

### Algorithm #1 - Hypothesis Testing

In [64]:
result_HT = run_HT(folder_path,files,startrow=839,normal_data=normal_data_df,nodes=nodes,edges_list=edges,key_nodes=check_nodes,colors=colors,pos=pos)

### Algorithm #2 - Epsilon Diagnosis

In [66]:
result_ED = run_ED(folder_path,files,startrow=839,normal_data=normal_data_df,nodes=nodes,edges_list=edges,key_nodes=check_nodes,colors=colors,pos=pos)

### Algorithm #3 - Random Walk

In [67]:
result_RW = run_RW(folder_path,files,startrow=839,normal_data=normal_data_df,nodes=nodes,edges_list=edges,key_nodes=check_nodes,colors=colors,pos=pos)

### Algorithm #4: RCD

In [68]:
result_RCD = run_RCD(folder_path,files,startrow=839,normal_data=normal_data_df,nodes=nodes,edges_list=edges,key_nodes=check_nodes,colors=colors,pos=pos)

## Data Analysis

### Summary Algorithms

In [79]:
result_all = pd.DataFrame()
result_all = pd.concat([result_HT,result_ED,result_RW,result_RCD])

result_all

,max_Vel_2-1,max_Vel_2-2,max_Vel_2-3,max_Vel_2-4,max_Vel_2-5,max_Vel_2-6,max_Vel_2-7,max_Vel_2-8,max_Vel_2-9,max_Vel_2-10,max_Vel_2-11,max_Vel_2-12,max_Vel_2-13,max_Vel_2-14,max_Vel_2-15,max_Vel_2-16
HT,"[rob_2_maxVel, rob_1_maxVel, rob_1_vacuum]","[rob_2_maxVel, rob_1_maxVel, rob_1_vacuum]","[rob_2_maxVel, rob_1_maxVel, rob_1_vacuum]","[rob_2_maxVel, rob_1_maxVel, rob_1_vacuum]","[rob_2_maxVel, rob_1_maxVel, rob_1_vacuum]","[rob_2_maxVel, rob_1_maxVel, rob_1_vacuum]","[rob_2_maxVel, rob_1_maxVel, rob_1_vacuum]","[rob_2_maxVel, rob_1_maxVel, rob_1_vacuum]","[rob_2_maxVel, rob_1_vacuum, rob_1_3]","[rob_2_maxVel, rob_1_maxVel, rob_1_vacuum]","[rob_2_maxVel, rob_1_vacuum, rob_1_1]","[rob_2_maxVel, rob_1_maxVel, rob_1_vacuum]","[rob_2_maxVel, rob_1_maxVel, rob_1_vacuum]","[rob_2_maxVel, rob_1_maxVel, rob_1_vacuum]","[rob_2_maxVel, rob_1_maxVel, rob_1_vacuum]","[rob_2_maxVel, rob_1_maxVel, rob_1_vacuum]"
ED,"[rob_1_vacuum, rob_1_4, rob_1_3]","[EoL_1_X, rob_1_vacuum, rob_1_4]","[rob_1_vacuum, rob_1_4, rob_1_3]","[rob_1_vacuum, rob_1_4, rob_1_3]","[cam_3_Y, EoL_1_X, rob_1_vacuum]","[rob_1_vacuum, rob_1_4, rob_1_3]","[rob_1_vacuum, rob_1_4, rob_1_3]","[rob_1_vacuum, rob_1_4, rob_1_3]","[rob_1_vacuum, rob_1_4, rob_1_3]","[rob_1_vacuum, rob_1_4, rob_1_3]","[cam_3_Y, EoL_1_X, rob_1_vacuum]","[cam_3_Y, rob_1_vacuum, rob_1_4]","[rob_1_vacuum, rob_1_4, rob_1_3]","[rob_1_vacuum, rob_1_4, rob_1_3]","[rob_1_vacuum, rob_1_4, rob_1_3]","[cam_3_Y, rob_1_vacuum, rob_1_4]"
RW,"[rob_2_maxVel, rob_2_supply, cam_1_Y]","[rob_2_maxVel, cam_1_X, rob_2_supply]","[rob_2_maxVel, cam_1_Y, rob_2_supply]","[rob_2_maxVel, con_2, rob_2_supply]","[rob_2_maxVel, rob_2_supply, rob_1_supply]","[rob_2_maxVel, cam_1_Y, rob_1_supply]","[rob_2_supply, rob_2_maxVel, cam_1_Y]","[rob_2_maxVel, rob_2_supply, cam_1_X]","[rob_2_maxVel, rob_2_supply, cam_1_Y]","[rob_2_supply, rob_2_maxVel, cam_1_Y]","[rob_2_maxVel, rob_2_supply, con_2]","[rob_2_maxVel, rob_2_supply, con_2]","[rob_2_supply, rob_2_maxVel, con_2]","[rob_2_supply, rob_2_maxVel, cam_1_X]","[rob_2_maxVel, rob_2_supply, cam_1_Y]","[rob_2_maxVel, rob_2_supply, cam_1_Y]"
RCD,"[rob_2_maxVel, EoL_6_Y]","[EoL_6_Y, EoL_6_X]",[EoL_6_X],[rob_2_maxVel],[rob_2_maxVel],[rob_2_maxVel],"[EoL_6_Y, EoL_6_X]",[rob_2_maxVel],[EoL_6_X],"[EoL_6_Y, rob_2_maxVel, EoL_6_X]",[EoL_6_X],"[EoL_6_Y, rob_2_maxVel]",[EoL_6_X],"[EoL_6_X, rob_2_maxVel, EoL_6_Y]","[EoL_6_Y, rob_2_maxVel]",[EoL_6_Y]


###  Intervention - Root Cause

In [70]:
abnormal_sets = pd.DataFrame()
abnormal_sets['size_1'] = ['cam_1_X', 'cam_1_Y']
abnormal_sets['feeder_3'] = ['cam_3_X', 'cam_3_Y']
abnormal_sets['gripper_1'] = ['rob_1_supply',None]
abnormal_sets['max_Vel_2'] = ['rob_2_maxVel',None]

### TOP 3  - Root Cause

In [81]:
result_data_3top = results_top_3(result_all,abnormal_sets)
result_data_3top.tail()

,HT,ED,RW,RCD
max_Vel_2-13,1.0,0.0,1.0,0.0000
max_Vel_2-14,1.0,0.0,1.0,1.0000
max_Vel_2-15,1.0,0.0,1.0,1.0000
max_Vel_2-16,1.0,0.0,1.0,0.0000
Total,1.0,0.0,1.0,0.5625


### TOP 1  - Root Cause

In [82]:
result_data_1top = results_top_1(result_all,abnormal_sets)
result_data_1top.tail()


,HT,ED,RW,RCD
max_Vel_2-13,1.0,0.0,0.00,0.0000
max_Vel_2-14,1.0,0.0,0.00,0.0000
max_Vel_2-15,1.0,0.0,1.00,0.0000
max_Vel_2-16,1.0,0.0,1.00,0.0000
Total,1.0,0.0,0.75,0.3125


## Variation 1 - Overlap

### HT -Algorithm

In [73]:
result_HT_overlap_1 = run_HT_overlap(folder_path,files,startrow=839,overlap_p=0.10,nodes=nodes,edges_list=edges,key_nodes=check_nodes,colors=colors,pos=pos)
result_HT_overlap_2 = run_HT_overlap(folder_path,files,startrow=839,overlap_p=0.20,nodes=nodes,edges_list=edges,key_nodes=check_nodes,colors=colors,pos=pos)
result_HT_overlap_3 = run_HT_overlap(folder_path,files,startrow=839,overlap_p=0.50,nodes=nodes,edges_list=edges,key_nodes=check_nodes,colors=colors,pos=pos)
result_HT_overlap_4 = run_HT_overlap(folder_path,files,startrow=839,overlap_p=0.75,nodes=nodes,edges_list=edges,key_nodes=check_nodes,colors=colors,pos=pos)
result_HT_overlap_5 = run_HT_overlap(folder_path,files,startrow=839,overlap_p=0.95,nodes=nodes,edges_list=edges,key_nodes=check_nodes,colors=colors,pos=pos)

NameError: name 'run_HT_overlap' is not defined

In [ ]:
result_overlap_HT = pd.DataFrame()
result_overlap_HT = pd.concat([result_HT_overlap_1,result_HT_overlap_2,result_HT_overlap_3,result_HT_overlap_4,result_HT_overlap_5], ignore_index=True)
result_overlap_HT['Overlap Percent'] = pd.DataFrame({'HT':['10%','20%','50%','75%','95%']})
result_overlap_HT.set_index('Overlap Percent', inplace=True)
result_overlap_HT

In [ ]:
result_data_3top = results_top_3(result_overlap_HT,abnormal_sets)
result_data_3top.tail()

In [ ]:
result_data_1top = results_top_1(result_overlap_HT,abnormal_sets)
result_data_1top.tail()

### ED - Algorithm

In [ ]:
result_ED_overlap_1 = run_ED_overlap(folder_path,files,startrow=839,overlap_p=0.10,nodes=nodes,edges_list=edges,key_nodes=check_nodes,colors=colors,pos=pos)
result_ED_overlap_2 = run_ED_overlap(folder_path,files,startrow=839,overlap_p=0.20,nodes=nodes,edges_list=edges,key_nodes=check_nodes,colors=colors,pos=pos)
result_ED_overlap_3 = run_ED_overlap(folder_path,files,startrow=839,overlap_p=0.50,nodes=nodes,edges_list=edges,key_nodes=check_nodes,colors=colors,pos=pos)
result_ED_overlap_4 = run_ED_overlap(folder_path,files,startrow=839,overlap_p=0.75,nodes=nodes,edges_list=edges,key_nodes=check_nodes,colors=colors,pos=pos)
result_ED_overlap_5 = run_ED_overlap(folder_path,files,startrow=839,overlap_p=0.95,nodes=nodes,edges_list=edges,key_nodes=check_nodes,colors=colors,pos=pos)

In [ ]:
result_overlap_ED = pd.DataFrame()
result_overlap_ED = pd.concat([result_ED_overlap_1,result_ED_overlap_2,result_ED_overlap_3,result_ED_overlap_4,result_ED_overlap_5], ignore_index=True)
result_overlap_ED['Overlap Percent'] = pd.DataFrame({'ED':['10%','20%','50%','75%','95%']})
result_overlap_ED.set_index('Overlap Percent', inplace=True)
result_overlap_ED

In [ ]:
result_data_3top = results_top_3(result_overlap_ED,abnormal_sets)
result_data_3top.tail()

In [ ]:
result_data_1top = results_top_1(result_overlap_ED,abnormal_sets)
result_data_1top.tail()

### RW - Algorithm

In [ ]:
result_RW_overlap_1 = run_RW_overlap(folder_path,files,startrow=839,overlap_p=0.10,nodes=nodes,edges_list=edges,key_nodes=check_nodes,colors=colors,pos=pos)
result_RW_overlap_2 = run_RW_overlap(folder_path,files,startrow=839,overlap_p=0.20,nodes=nodes,edges_list=edges,key_nodes=check_nodes,colors=colors,pos=pos)
result_RW_overlap_3 = run_RW_overlap(folder_path,files,startrow=839,overlap_p=0.50,nodes=nodes,edges_list=edges,key_nodes=check_nodes,colors=colors,pos=pos)
result_RW_overlap_4 = run_RW_overlap(folder_path,files,startrow=839,overlap_p=0.75,nodes=nodes,edges_list=edges,key_nodes=check_nodes,colors=colors,pos=pos)
result_RW_overlap_5 = run_RW_overlap(folder_path,files,startrow=839,overlap_p=0.95,nodes=nodes,edges_list=edges,key_nodes=check_nodes,colors=colors,pos=pos)

In [ ]:
result_overlap_RW = pd.DataFrame()
result_overlap_RW = pd.concat([result_RW_overlap_1,result_RW_overlap_2,result_RW_overlap_3,result_RW_overlap_4,result_RW_overlap_5], ignore_index=True)
result_overlap_RW['Overlap Percent'] = pd.DataFrame({'RW':['10%','20%','50%','75%','95%']})
result_overlap_RW.set_index('Overlap Percent', inplace=True)
result_overlap_RW

In [ ]:
result_data_3top = results_top_3(result_overlap_RW,abnormal_sets)
result_data_3top.tail()

In [ ]:
result_data_1top = results_top_1(result_overlap_RW,abnormal_sets)
result_data_1top.tail()

### RCD - Algorithm

In [ ]:
result_RCD_overlap_1 = run_RCD_overlap(folder_path,files,startrow=839,overlap_p=0.10,nodes=nodes,edges_list=edges,key_nodes=check_nodes,colors=colors,pos=pos)
result_RCD_overlap_2 = run_RCD_overlap(folder_path,files,startrow=839,overlap_p=0.20,nodes=nodes,edges_list=edges,key_nodes=check_nodes,colors=colors,pos=pos)
result_RCD_overlap_3 = run_RCD_overlap(folder_path,files,startrow=839,overlap_p=0.50,nodes=nodes,edges_list=edges,key_nodes=check_nodes,colors=colors,pos=pos)
result_RCD_overlap_4 = run_RCD_overlap(folder_path,files,startrow=839,overlap_p=0.75,nodes=nodes,edges_list=edges,key_nodes=check_nodes,colors=colors,pos=pos)
result_RCD_overlap_5 = run_RCD_overlap(folder_path,files,startrow=839,overlap_p=0.95,nodes=nodes,edges_list=edges,key_nodes=check_nodes,colors=colors,pos=pos)

In [ ]:
result_overlap_RCD = pd.DataFrame()
result_overlap_RCD = pd.concat([result_RCD_overlap_1,result_RCD_overlap_2,result_RCD_overlap_3,result_RCD_overlap_4,result_RCD_overlap_5], ignore_index=True)
result_overlap_RCD['Overlap Percent'] = pd.DataFrame({'RCD':['10%','20%','50%','75%','95%']})
result_overlap_RCD.set_index('Overlap Percent', inplace=True)
result_overlap_RCD

In [ ]:
result_data_3top = results_top_3(result_overlap_RCD,abnormal_sets)
result_data_3top.tail()

In [ ]:
result_data_1top = results_top_1(result_overlap_RCD,abnormal_sets)
result_data_1top.tail()

## Variation 2 - Normal Size

The ED-algorithm requires the same length for the normal and abnormal data set.

### HT - Algorithm

In [ ]:
result_HT_normal_size_1 = run_HT_normal_size(folder_path,files,startrow=839,normal_size_p=2.00,nodes=nodes,edges_list=edges,key_nodes=check_nodes,colors=colors,pos=pos)
result_HT_normal_size_2 = run_HT_normal_size(folder_path,files,startrow=839,normal_size_p=1.5,nodes=nodes,edges_list=edges,key_nodes=check_nodes,colors=colors,pos=pos)
result_HT_normal_size_3 = run_HT_normal_size(folder_path,files,startrow=839,normal_size_p=1.0,nodes=nodes,edges_list=edges,key_nodes=check_nodes,colors=colors,pos=pos)
result_HT_normal_size_4 = run_HT_normal_size(folder_path,files,startrow=839,normal_size_p=0.5,nodes=nodes,edges_list=edges,key_nodes=check_nodes,colors=colors,pos=pos)
result_HT_normal_size_5 = run_HT_normal_size(folder_path,files,startrow=839,normal_size_p=0.2,nodes=nodes,edges_list=edges,key_nodes=check_nodes,colors=colors,pos=pos)

In [ ]:
result_normal_size_HT = pd.DataFrame()
result_normal_size_HT = pd.concat([result_HT_normal_size_1,result_HT_normal_size_2,result_HT_normal_size_3,result_HT_normal_size_4,result_HT_normal_size_5], ignore_index=True)
result_normal_size_HT['Normal Size Percent'] = pd.DataFrame({'HT':['200%','150%','100%','50%','20%']})
result_normal_size_HT.set_index('Normal Size Percent', inplace=True)
result_normal_size_HT

In [ ]:
result_data_3top = results_top_3(result_normal_size_HT,abnormal_sets)
result_data_3top.tail()

In [ ]:
result_data_1top = results_top_1(result_normal_size_HT,abnormal_sets)
result_data_1top.tail()

### RW - Algorithm

In [ ]:
result_RW_normal_size_1 = run_RW_normal_size(folder_path,files,startrow=839,normal_size_p=2.00,nodes=nodes,edges_list=edges,key_nodes=check_nodes,colors=colors,pos=pos)
result_RW_normal_size_2 = run_RW_normal_size(folder_path,files,startrow=839,normal_size_p=1.5,nodes=nodes,edges_list=edges,key_nodes=check_nodes,colors=colors,pos=pos)
result_RW_normal_size_3 = run_RW_normal_size(folder_path,files,startrow=839,normal_size_p=1.0,nodes=nodes,edges_list=edges,key_nodes=check_nodes,colors=colors,pos=pos)
result_RW_normal_size_4 = run_RW_normal_size(folder_path,files,startrow=839,normal_size_p=0.5,nodes=nodes,edges_list=edges,key_nodes=check_nodes,colors=colors,pos=pos)
result_RW_normal_size_5 = run_RW_normal_size(folder_path,files,startrow=839,normal_size_p=0.2,nodes=nodes,edges_list=edges,key_nodes=check_nodes,colors=colors,pos=pos)

In [ ]:
result_normal_size_RW = pd.DataFrame()
result_normal_size_RW = pd.concat([result_RW_normal_size_1,result_RW_normal_size_2,result_RW_normal_size_3,result_RW_normal_size_4,result_RW_normal_size_5], ignore_index=True)
result_normal_size_RW['Normal Size Percent'] = pd.DataFrame({'RW':['200%','150%','100%','50%','20%']})
result_normal_size_RW.set_index('Normal Size Percent', inplace=True)
result_normal_size_RW

In [ ]:
result_data_3top = results_top_3(result_normal_size_RW,abnormal_sets)
result_data_3top.tail()

In [ ]:
result_data_1top = results_top_1(result_normal_size_RW,abnormal_sets)
result_data_1top.tail()

### RCD - Algorithm

In [ ]:
result_RCD_normal_size_1 = run_RCD_normal_size(folder_path,files,startrow=839,normal_size_p=2.00,nodes=nodes,edges_list=edges,key_nodes=check_nodes,colors=colors,pos=pos)
result_RCD_normal_size_2 = run_RCD_normal_size(folder_path,files,startrow=839,normal_size_p=1.5,nodes=nodes,edges_list=edges,key_nodes=check_nodes,colors=colors,pos=pos)
result_RCD_normal_size_3 = run_RCD_normal_size(folder_path,files,startrow=839,normal_size_p=1.0,nodes=nodes,edges_list=edges,key_nodes=check_nodes,colors=colors,pos=pos)
result_RCD_normal_size_4 = run_RCD_normal_size(folder_path,files,startrow=839,normal_size_p=0.5,nodes=nodes,edges_list=edges,key_nodes=check_nodes,colors=colors,pos=pos)
result_RCD_normal_size_5 = run_RCD_normal_size(folder_path,files,startrow=839,normal_size_p=0.2,nodes=nodes,edges_list=edges,key_nodes=check_nodes,colors=colors,pos=pos)


In [ ]:
result_normal_size_RCD = pd.DataFrame()
result_normal_size_RCD = pd.concat([result_RCD_normal_size_1,result_RCD_normal_size_2,result_RCD_normal_size_3,result_RCD_normal_size_4,result_RCD_normal_size_5], ignore_index=True)
result_normal_size_RCD['Normal Size Percent'] = pd.DataFrame({'RCD':['200%','150%','100%','50%','20%']})
result_normal_size_RCD.set_index('Normal Size Percent', inplace=True)
result_normal_size_RCD

In [ ]:
result_data_3top = results_top_3(result_normal_size_RCD,abnormal_sets)
result_data_3top.tail()

In [ ]:
result_data_1top = results_top_1(result_normal_size_RCD,abnormal_sets)
result_data_1top.tail()

## Variation 3 - Abnormal Size

The ED-algorithm requires the same length for the normal and abnormal data set.

### HT - Algorithm

In [ ]:
result_HT_abnormal_size_1 = run_HT_abnormal_size(folder_path,files,startrow=839,abnormal_size_p=2.00,nodes=nodes,edges_list=edges,key_nodes=check_nodes,colors=colors,pos=pos)
result_HT_abnormal_size_2 = run_HT_abnormal_size(folder_path,files,startrow=839,abnormal_size_p=1.5,nodes=nodes,edges_list=edges,key_nodes=check_nodes,colors=colors,pos=pos)
result_HT_abnormal_size_3 = run_HT_abnormal_size(folder_path,files,startrow=839,abnormal_size_p=1.0,nodes=nodes,edges_list=edges,key_nodes=check_nodes,colors=colors,pos=pos)
result_HT_abnormal_size_4 = run_HT_abnormal_size(folder_path,files,startrow=839,abnormal_size_p=0.5,nodes=nodes,edges_list=edges,key_nodes=check_nodes,colors=colors,pos=pos)
result_HT_abnormal_size_5 = run_HT_abnormal_size(folder_path,files,startrow=839,abnormal_size_p=0.2,nodes=nodes,edges_list=edges,key_nodes=check_nodes,colors=colors,pos=pos)

In [ ]:
result_abnormal_size_HT = pd.DataFrame()
result_abnormal_size_HT = pd.concat([result_HT_abnormal_size_1,result_HT_abnormal_size_2,result_HT_abnormal_size_3,result_HT_abnormal_size_4,result_HT_abnormal_size_5], ignore_index=True)
result_abnormal_size_HT['abnormal Size Percent'] = pd.DataFrame({'HT':['200%','150%','100%','50%','20%']})
result_abnormal_size_HT.set_index('abnormal Size Percent', inplace=True)
result_abnormal_size_HT

In [ ]:
result_data_3top = results_top_3(result_abnormal_size_HT,abnormal_sets)
result_data_3top.tail()

In [ ]:
result_data_1top = results_top_1(result_abnormal_size_HT,abnormal_sets)
result_data_1top.tail()

### RW - Algorithm

In [ ]:
result_RW_abnormal_size_1 = run_RW_abnormal_size(folder_path,files,startrow=839,abnormal_size_p=2.00,nodes=nodes,edges_list=edges,key_nodes=check_nodes,colors=colors,pos=pos)
result_RW_abnormal_size_2 = run_RW_abnormal_size(folder_path,files,startrow=839,abnormal_size_p=1.5,nodes=nodes,edges_list=edges,key_nodes=check_nodes,colors=colors,pos=pos)
result_RW_abnormal_size_3 = run_RW_abnormal_size(folder_path,files,startrow=839,abnormal_size_p=1.0,nodes=nodes,edges_list=edges,key_nodes=check_nodes,colors=colors,pos=pos)
result_RW_abnormal_size_4 = run_RW_abnormal_size(folder_path,files,startrow=839,abnormal_size_p=0.5,nodes=nodes,edges_list=edges,key_nodes=check_nodes,colors=colors,pos=pos)
result_RW_abnormal_size_5 = run_RW_abnormal_size(folder_path,files,startrow=839,abnormal_size_p=0.2,nodes=nodes,edges_list=edges,key_nodes=check_nodes,colors=colors,pos=pos)

In [ ]:
result_abnormal_size_RW = pd.DataFrame()
result_abnormal_size_RW = pd.concat([result_RW_abnormal_size_1,result_RW_abnormal_size_2,result_RW_abnormal_size_3,result_RW_abnormal_size_4,result_RW_abnormal_size_5], ignore_index=True)
result_abnormal_size_RW['abnormal Size Percent'] = pd.DataFrame({'RW':['200%','150%','100%','50%','20%']})
result_abnormal_size_RW.set_index('abnormal Size Percent', inplace=True)
result_abnormal_size_RW

In [ ]:
result_data_3top = results_top_3(result_abnormal_size_RW,abnormal_sets)
result_data_3top.tail()

In [ ]:
result_data_1top = results_top_1(result_abnormal_size_RW,abnormal_sets)
result_data_1top.tail()

### RCD - Algorithm

In [ ]:
result_RCD_abnormal_size_1 = run_RCD_abnormal_size(folder_path,files,startrow=839,abnormal_size_p=2.00,nodes=nodes,edges_list=edges,key_nodes=check_nodes,colors=colors,pos=pos)
result_RCD_abnormal_size_2 = run_RCD_abnormal_size(folder_path,files,startrow=839,abnormal_size_p=1.5,nodes=nodes,edges_list=edges,key_nodes=check_nodes,colors=colors,pos=pos)
result_RCD_abnormal_size_3 = run_RCD_abnormal_size(folder_path,files,startrow=839,abnormal_size_p=1.0,nodes=nodes,edges_list=edges,key_nodes=check_nodes,colors=colors,pos=pos)
result_RCD_abnormal_size_4 = run_RCD_abnormal_size(folder_path,files,startrow=839,abnormal_size_p=0.5,nodes=nodes,edges_list=edges,key_nodes=check_nodes,colors=colors,pos=pos)
result_RCD_abnormal_size_5 = run_RCD_abnormal_size(folder_path,files,startrow=839,abnormal_size_p=0.2,nodes=nodes,edges_list=edges,key_nodes=check_nodes,colors=colors,pos=pos)


In [ ]:
result_abnormal_size_RCD = pd.DataFrame()
result_abnormal_size_RCD = pd.concat([result_RCD_abnormal_size_1,result_RCD_abnormal_size_2,result_RCD_abnormal_size_3,result_RCD_abnormal_size_4,result_RCD_abnormal_size_5], ignore_index=True)
result_abnormal_size_RCD['abnormal Size Percent'] = pd.DataFrame({'RCD':['200%','150%','100%','50%','20%']})
result_abnormal_size_RCD.set_index('abnormal Size Percent', inplace=True)
result_abnormal_size_RCD

In [ ]:
result_data_3top = results_top_3(result_abnormal_size_RCD,abnormal_sets)
result_data_3top.tail()

In [ ]:
result_data_1top = results_top_1(result_abnormal_size_RCD,abnormal_sets)
result_data_1top.tail()

## Variation 4 - Causal Graph, adjacency matrix

Only the HT- and RW-Algorithm use an adjacency matrix for training of each model.

### HT - Algorithm

In [ ]:
result_HT_edges_1 = run_HT_edges_delete(folder_path,files,startrow=839,edges_delete_n=20,nodes=nodes,edges_list=edges,key_nodes=check_nodes,colors=colors,pos=pos)
result_HT_edges_2 = run_HT_edges_delete(folder_path,files,startrow=839,edges_delete_n=40,nodes=nodes,edges_list=edges,key_nodes=check_nodes,colors=colors,pos=pos)
result_HT_edges_3 = run_HT_edges_delete(folder_path,files,startrow=839,edges_delete_n=60,nodes=nodes,edges_list=edges,key_nodes=check_nodes,colors=colors,pos=pos)
result_HT_edges_4 = run_HT_edges_delete(folder_path,files,startrow=839,edges_delete_n=80,nodes=nodes,edges_list=edges,key_nodes=check_nodes,colors=colors,pos=pos)
result_HT_edges_5 = run_HT_edges_delete(folder_path,files,startrow=839,edges_delete_n=100,nodes=nodes,edges_list=edges,key_nodes=check_nodes,colors=colors,pos=pos)

In [ ]:
result_edges_HT = pd.DataFrame()
result_edges_HT = pd.concat([result_HT_edges_1,result_HT_edges_2,result_HT_edges_3,result_HT_edges_4,result_HT_edges_5], ignore_index=True)
result_edges_HT['Edges Missing'] = pd.DataFrame({'HT':['20','40','60','80','100']})
result_edges_HT.set_index('Edges Missing', inplace=True)
result_edges_HT

In [ ]:
result_data_3top = results_top_3(result_edges_HT,abnormal_sets)
result_data_3top.tail()

In [ ]:
result_data_1top = results_top_1(result_edges_HT,abnormal_sets)
result_data_1top.tail()

### RW - Algorithm

In [ ]:
result_RW_edges_1 = run_RW_edges_delete(folder_path,files,startrow=839,edges_delete_n=20,nodes=nodes,edges_list=edges,key_nodes=check_nodes,colors=colors,pos=pos)
result_RW_edges_2 = run_RW_edges_delete(folder_path,files,startrow=839,edges_delete_n=40,nodes=nodes,edges_list=edges,key_nodes=check_nodes,colors=colors,pos=pos)
result_RW_edges_3 = run_RW_edges_delete(folder_path,files,startrow=839,edges_delete_n=60,nodes=nodes,edges_list=edges,key_nodes=check_nodes,colors=colors,pos=pos)
result_RW_edges_4 = run_RW_edges_delete(folder_path,files,startrow=839,edges_delete_n=80,nodes=nodes,edges_list=edges,key_nodes=check_nodes,colors=colors,pos=pos)
result_RW_edges_5 = run_RW_edges_delete(folder_path,files,startrow=839,edges_delete_n=100,nodes=nodes,edges_list=edges,key_nodes=check_nodes,colors=colors,pos=pos)

In [ ]:
result_edges_RW = pd.DataFrame()
result_edges_RW = pd.concat([result_RW_edges_1,result_RW_edges_2,result_RW_edges_3,result_RW_edges_4,result_RW_edges_5], ignore_index=True)
result_edges_RW['Edges Missing'] = pd.DataFrame({'RW':['20','40','60','80','100']})
result_edges_RW.set_index('Edges Missing', inplace=True)
result_edges_RW

In [ ]:
result_data_3top = results_top_3(result_edges_RW,abnormal_sets)
result_data_3top.tail()

In [ ]:
result_data_1top = results_top_1(result_edges_RW,abnormal_sets)
result_data_1top.tail()

## Variation 5 - Size

### HT -Algorithm

In [ ]:
result_HT_size_1 = run_HT_size(folder_path,files,startrow=839,size_p=2,nodes=nodes,edges_list=edges,key_nodes=check_nodes,colors=colors,pos=pos)
result_HT_size_2 = run_HT_size(folder_path,files,startrow=839,size_p=1.5,nodes=nodes,edges_list=edges,key_nodes=check_nodes,colors=colors,pos=pos)
result_HT_size_3 = run_HT_size(folder_path,files,startrow=839,size_p=1.0,nodes=nodes,edges_list=edges,key_nodes=check_nodes,colors=colors,pos=pos)
result_HT_size_4 = run_HT_size(folder_path,files,startrow=839,size_p=0.5,nodes=nodes,edges_list=edges,key_nodes=check_nodes,colors=colors,pos=pos)
result_HT_size_5 = run_HT_size(folder_path,files,startrow=839,size_p=0.2,nodes=nodes,edges_list=edges,key_nodes=check_nodes,colors=colors,pos=pos)

In [ ]:
result_size_HT = pd.DataFrame()
result_size_HT = pd.concat([result_HT_size_1,result_HT_size_2,result_HT_size_3,result_HT_size_4,result_HT_size_5], ignore_index=True)
result_size_HT['Data Size'] = pd.DataFrame({'HT':['200%','150%','100%','50%','20%']})
result_size_HT.set_index('Data Size', inplace=True)
result_size_HT

In [ ]:
result_data_3top = results_top_3(result_size_HT,abnormal_sets)
result_data_3top.tail()

In [ ]:
result_data_1top = results_top_1(result_size_HT,abnormal_sets)
result_data_1top.tail()

### ED - Algorithm

In [ ]:
result_ED_size_1 = run_ED_size(folder_path,files,startrow=839,size_p=2,nodes=nodes,edges_list=edges,key_nodes=check_nodes,colors=colors,pos=pos)
result_ED_size_2 = run_ED_size(folder_path,files,startrow=839,size_p=1.5,nodes=nodes,edges_list=edges,key_nodes=check_nodes,colors=colors,pos=pos)
result_ED_size_3 = run_ED_size(folder_path,files,startrow=839,size_p=1.0,nodes=nodes,edges_list=edges,key_nodes=check_nodes,colors=colors,pos=pos)
result_ED_size_4 = run_ED_size(folder_path,files,startrow=839,size_p=0.5,nodes=nodes,edges_list=edges,key_nodes=check_nodes,colors=colors,pos=pos)
result_ED_size_5 = run_ED_size(folder_path,files,startrow=839,size_p=0.20,nodes=nodes,edges_list=edges,key_nodes=check_nodes,colors=colors,pos=pos)

In [ ]:
result_size_ED = pd.DataFrame()
result_size_ED = pd.concat([result_ED_size_1,result_ED_size_2,result_ED_size_3,result_ED_size_4,result_ED_size_5], ignore_index=True)
result_size_ED['Data Size'] = pd.DataFrame({'ED':['200%','150%','100%','50%','20%']})
result_size_ED.set_index('Data Size', inplace=True)
result_size_ED

In [ ]:
result_data_3top = results_top_3(result_size_ED,abnormal_sets)
result_data_3top.tail()

In [ ]:
result_data_1top = results_top_1(result_size_ED,abnormal_sets)
result_data_1top.tail()

### RW - Algorithm

In [ ]:
result_RW_size_1 = run_RW_size(folder_path,files,startrow=839,size_p=2,nodes=nodes,edges_list=edges,key_nodes=check_nodes,colors=colors,pos=pos)
result_RW_size_2 = run_RW_size(folder_path,files,startrow=839,size_p=1.5,nodes=nodes,edges_list=edges,key_nodes=check_nodes,colors=colors,pos=pos)
result_RW_size_3 = run_RW_size(folder_path,files,startrow=839,size_p=1.0,nodes=nodes,edges_list=edges,key_nodes=check_nodes,colors=colors,pos=pos)
result_RW_size_4 = run_RW_size(folder_path,files,startrow=839,size_p=0.5,nodes=nodes,edges_list=edges,key_nodes=check_nodes,colors=colors,pos=pos)
result_RW_size_5 = run_RW_size(folder_path,files,startrow=839,size_p=0.20,nodes=nodes,edges_list=edges,key_nodes=check_nodes,colors=colors,pos=pos)

In [ ]:
result_size_RW = pd.DataFrame()
result_size_RW = pd.concat([result_RW_size_1,result_RW_size_2,result_RW_size_3,result_RW_size_4,result_RW_size_5], ignore_index=True)
result_size_RW['Data Size'] = pd.DataFrame({'RW':['200%','150%','100%','50%','20%']})
result_size_RW.set_index('Data Size', inplace=True)
result_size_RW

In [ ]:
result_data_3top = results_top_3(result_size_RW,abnormal_sets)
result_data_3top.tail()

In [ ]:
result_data_1top = results_top_1(result_size_RW,abnormal_sets)
result_data_1top.tail()

### RCD - Algorithm

In [ ]:
result_RCD_size_1 = run_RCD_size(folder_path,files,startrow=839,size_p=2,nodes=nodes,edges_list=edges,key_nodes=check_nodes,colors=colors,pos=pos)
result_RCD_size_2 = run_RCD_size(folder_path,files,startrow=839,size_p=1.5,nodes=nodes,edges_list=edges,key_nodes=check_nodes,colors=colors,pos=pos)
result_RCD_size_3 = run_RCD_size(folder_path,files,startrow=839,size_p=1.0,nodes=nodes,edges_list=edges,key_nodes=check_nodes,colors=colors,pos=pos)
result_RCD_size_4 = run_RCD_size(folder_path,files,startrow=839,size_p=0.5,nodes=nodes,edges_list=edges,key_nodes=check_nodes,colors=colors,pos=pos)
result_RCD_size_5 = run_RCD_size(folder_path,files,startrow=839,size_p=0.20,nodes=nodes,edges_list=edges,key_nodes=check_nodes,colors=colors,pos=pos)

In [ ]:
result_size_RCD = pd.DataFrame()
result_size_RCD = pd.concat([result_RCD_size_1,result_RCD_size_2,result_RCD_size_3,result_RCD_size_4,result_RCD_size_5], ignore_index=True)
result_size_RCD['Data Size'] = pd.DataFrame({'RCD':['200%','150%','100%','50%','20%']})
result_size_RCD.set_index('Data Size', inplace=True)
result_size_RCD

In [ ]:
result_data_3top = results_top_3(result_size_RCD,abnormal_sets)
result_data_3top.tail()

In [ ]:
result_data_1top = results_top_1(result_size_RCD,abnormal_sets)
result_data_1top.tail()